In [3]:
import napari
import os
from tqdm import tqdm
import numpy as np
from ultrack import MainConfig, load_config, track, to_tracks_layer, tracks_to_zarr
import glob2 as glob
import zarr
import json
import skimage.io as io
import pandas as pd
import umap.umap_ as umap
from sklearn.preprocessing import StandardScaler

# # set parameters
root = "E:\\Nick\\Cole Trapnell's Lab Dropbox\\Nick Lammers\\Nick\\killi_tracker\\"
project_name = "231016_EXP40_LCP1_UVB_300mJ_WT_Timelapse_Raw"
image_dir = os.path.join(root, "built_data", project_name, "")
ds_factor = 2
config_name = "tracking_final.txt"
tracking_folder = config_name.replace(".txt", "")
tracking_folder = tracking_folder.replace(".toml", "")

tracking_directory = os.path.join(root, "built_data", "tracking", tracking_folder, project_name)
snip_directory = os.path.join(root, "built_data", "shape_snips", tracking_folder, project_name)

# load metadata
metadata_file_path = os.path.join(root, "metadata", project_name, "metadata.json")
f = open(metadata_file_path)
metadata = json.load(f)
scale_vec = np.asarray([metadata["ProbPhysicalSizeZ"], metadata["ProbPhysicalSizeY"], metadata["ProbPhysicalSizeX"]])

In [9]:
cell_shape_df = pd.read_csv(os.path.join(tracking_directory, "cell_shape_df.csv"))
cell_shape_df.head()
n_components = 2

np.random.seed(356)

shape_array = cell_shape_df.iloc[:, 2:].to_numpy()

reducer = umap.UMAP(n_components=n_components)
# scaled_shape_array = StandardScaler().fit_transform(shape_array)
shape_embedding = reducer.fit_transform(shape_array)

cell_shape_df["UMAP00"] = shape_embedding[:, 0]
cell_shape_df["UMAP01"] = shape_embedding[:, 1]

In [23]:
import plotly.express as px
fig = px.scatter(cell_shape_df.loc[0:100, :], x="UMAP00", y="UMAP01", color="t", opacity=1)
fig.show()

In [24]:
from pyefd import elliptic_fourier_descriptors
import pyefd

shape_i = 100

print(cell_shape_df.loc[shape_i, ["UMAP00", "UMAP01"]])

shape_arr = np.reshape(np.concatenate(([1, 0, 0], shape_array[shape_i, :])), (10, 4))

out = pyefd.reconstruct_contour(shape_arr)

fig = px.scatter(x=out[:,0], y=out[:, 1])
fig.show()

UMAP00    8.210272
UMAP01    8.396723
Name: 100, dtype: float64


In [18]:
out.shape

(300, 2)

In [15]:
shape_vec

array([ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00, -3.42739171e-01,
       -1.43980721e-02, -3.13172730e-02,  3.57292833e-04, -4.38622113e-03,
        9.78371922e-02,  2.15180417e-03,  1.80885125e-02, -6.86050915e-02,
       -1.16295776e-02,  1.43943268e-03, -4.39375702e-02, -1.05192813e-02,
        2.00713932e-02,  3.88804463e-03, -1.74191921e-02, -6.93419981e-02,
       -6.98573670e-03,  7.44990037e-03, -4.64176264e-03, -1.26854655e-02,
       -8.96446506e-04,  6.97728643e-03,  9.95176696e-03,  4.45215362e-03,
       -8.89890558e-03,  3.43192473e-03,  5.66922790e-04,  1.49860644e-02,
        2.23052874e-03,  1.76684207e-03,  1.62788800e-03,  5.12475840e-03,
       -1.10758518e-03,  1.08874582e-02,  5.92583223e-03, -2.64891884e-03])